In [1]:
import pandas as pd
import numpy as np
import networkx as nx
# !pip install gensim==4.3.0
# !pip install smart_open==1.9.0
from gensim.models import Word2Vec
import os
import sys

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, median_absolute_error, mean_absolute_percentage_error

# Задание - Предсказание уровня экспресси белка

<img src='https://www.researchgate.net/publication/313504607/figure/fig3/AS:459880453677066@1486655453033/Protein-protein-interaction-PPI-network-of-DEGs-by-STRING-The-interaction-score-was.png'>




<div class="alert alert-info">
<b>Про биологию</b>
    
Экспрессия — процесс, в ходе которого наследственная информация от гена (последовательности нуклеотидов ДНК) преобразуется в функциональный продукт — белок. Уровнем экспрессии называют - количество белка, производящегося в этом процессе. Чем выше экспрессия белка, тем большее количество этого белка появляется в клетках человека. 
    
    

<div class="alert alert-info">    
<b>Важность задачи</b>
    
Существует множество причин необходимости в знании уровня экспресии белка. Например - это позволяет ученым разрабатывать лекарственные средства и оптимизировать их разработку. Теперь вам предстоит побыть в роли биоинформатика и помочь науке!
    
</div>


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой граф взаимойдествия белков. Где узлы это белки, взаимодействие между белками это ребро. 

Для каждого белка известен уровень его экспрессии. Ниже приведен список ребер `edges`. Информация по экспрессии белков, разбитая на `train` и `test`.
   
    
</div>

In [2]:
#Список ребер графа 
edges = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/edges.csv", sep=",") # Подгрузим данные
edges.head()

,node_1,node_2
0,344,50
1,344,153
2,344,532
3,344,679
4,344,986


In [3]:
#Подгрузим тренирочную выборку
train = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/train.csv", sep=",") # Подгрузим данные
train.head()

,target,node
0,0.251968,11142
1,0.689541,2243
2,0.678245,15514
3,0.272500,20944
4,0.248888,8721


In [4]:
# Подгрузим отложенную выборку для валидации
test = pd.read_csv("https://raw.githubusercontent.com/a-milenkin/Otus_HW_protein_expression/main/test.csv", sep=",")
test.head()

,target,node
0,0.279231,817
1,0.380795,9574
2,0.686527,1607
3,0.303594,4782
4,0.367374,24125


<div class="alert alert-info">
<b>Про Задачу</b>
    
Вам предлагается предсказать экспрессию белков (`target`) по приведенным данным для отложенной выборки. Ответы в отложенной выборке `test` даны вам для самостоятельной валидации.


    
   
    

<div class="alert alert-info">
<b>Замечание и комментарии</b>
    
    

По ряду причин датасет был упрощен так, чтобы выполнялись следующие условия:
* у графа одна компонента связанности. 
* удалены слишком крупные хабы
* плотность связей графа уменьшена
* решить задачу можно классическими ML подходами
    
   

<div class="alert alert-info">
<b>Оценка результатов</b>
    


Оценка точности модели будет оцениваться по метрике MSE на отложенной выборке `test`
        
</div>

<div class="alert alert-info">
<b>Автор задачи</b>

По всем дополнительным вопросами писать Александру Миленькину
* Телеграмм: Alerin75infskin
* Почта: milenkin.aa@phystech.edu
        
</div>

In [5]:
g = nx.from_pandas_edgelist(edges, "node_1", "node_2", create_using=nx.Graph())

In [6]:
# nx.draw_networkx(g) # отрисовка оч долгая

# Feature generation
## Показатели нодов  
### Центральность (centrality) - степень важности вершины
центральным можно считать того, кто связан с наибольшим числом членов группы. DС - это отношение количества связей определенного узла (в нашем случае журнала) к общему количеству других узлов.

In [7]:
%%time
degree_centrality = nx.degree_centrality(g)

CPU times: total: 15.6 ms
Wall time: 9 ms


### Betweenness centrality (Степень посредничества)
позволяет ранжировать вершины в графе, то есть оценить их важность с точки зрения количества проходящих через них путей.

In [8]:
%%time
betweenness_centrality = nx.betweenness_centrality(g, k=20)

CPU times: total: 7.41 s
Wall time: 7.4 s


### Closeness centrality (Степень близости)
Центральность по близости выражает, насколько близко узел расположен к остальным узлам сети. 

In [9]:
# Расчет слишком долгий

# %%time
# closeness_centrality = nx.closeness_centrality(g)

## Фитчи из топологии
### Word2Vec

In [10]:
!git clone https://github.com/stellargraph/stellargraph.git

cwd = os.getcwd() 
sys.path.append(os.path.join(cwd, 'stellargraph'))
# from stellargraph import StellarGraph
import stellargraph

fatal: destination path 'stellargraph' already exists and is not an empty directory.


In [11]:
g_stellar = stellargraph.StellarGraph.from_networkx(g)

In [12]:
rw = stellargraph.data.BiasedRandomWalk(g_stellar)

In [13]:
%%time
walks = rw.run(nodes=list(g_stellar.nodes()), length=10, n=3, p=0.4)

CPU times: total: 41.5 s
Wall time: 41.6 s


In [14]:
print(len(walks))

30000


In [15]:
# Конвертируем список узлов в список строк:
str_walks = [[str(a) for a in b] for b in walks]
for sw in str_walks[:10]:
    print(sw)

['344', '1300', '24', '13891', '24660', '2', '4734', '544', '8516', '19945']
['344', '8271', '100', '945', '100', '44', '181', '5466', '2518', '9960']
['344', '21679', '9411', '16804', '2661', '12346', '23462', '8349', '20051', '7929']
['50', '20377', '15292', '352', '13288', '4547', '26232', '388', '24617', '1507']
['50', '456', '15562', '2392', '15589', '26681', '452', '19189', '25022', '3911']
['50', '1188', '2913', '6906', '11097', '27802', '432', '24669', '508', '23072']
['153', '20417', '11173', '14', '14903', '1318', '17679', '18968', '278', '5832']
['153', '440', '836', '505', '15298', '21433', '241', '3170', '21779', '7616']
['153', '2488', '2849', '1206', '27479', '19671', '768', '10993', '562', '2030']
['532', '5391', '3406', '13417', '120', '15804', '27073', '2583', '6896', '22960']


In [16]:
model = Word2Vec(str_walks, vector_size=50, workers=4, min_count=3)

In [17]:
# Вот так выглядят наши эмбеддинги:
model.wv[g_stellar.nodes()[0]]

array([-2.3279732e-01,  2.2022559e-03,  8.2949013e-02,  1.0683715e-01,
       -3.2948771e-01, -3.1234956e-01,  5.6445718e-01,  8.8343304e-01,
       -9.6947592e-01,  5.2672264e-04, -1.7715904e-01, -7.9251224e-01,
       -2.9377037e-01,  5.6476128e-01, -2.9942995e-01, -8.6892180e-02,
        2.7951896e-01, -4.5116955e-01, -3.4779409e-01, -5.8686954e-01,
        2.7442357e-01,  6.4704669e-01,  1.2447059e+00, -1.7551093e-01,
        3.9187875e-01,  5.9922606e-01, -3.3549303e-01, -2.0645170e-01,
       -8.7135786e-01,  1.0058331e-01,  2.3729806e-01,  8.6324893e-02,
       -3.0081478e-01,  8.9915723e-02, -2.6642364e-01,  3.3684826e-01,
        1.6625388e-01,  6.1125819e-02,  4.1508964e-01, -6.7379338e-01,
        4.0671805e-01, -4.8334181e-01, -1.2951881e-01,  1.9625211e-01,
        1.6038163e+00, -3.4384787e-02, -3.6639738e-01, -5.0618529e-01,
        3.5105532e-01,  7.7846366e-01], dtype=float32)

### Собираем фитчи в один датафрейм

In [18]:
data0 = pd.DataFrame(np.stack([model.wv[str(i)] for i in g_stellar.nodes()]))
data0['node'] = g_stellar.nodes()
data0

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,node
0,-0.116137,0.004499,0.037470,0.056035,-0.162583,-0.157272,0.327033,0.484222,-0.555073,-0.002476,...,-0.245420,-0.045952,0.120820,0.883743,-0.026473,-0.215686,-0.286399,0.167177,0.429504,344
1,-0.205691,-0.009839,0.072909,0.066616,-0.257985,-0.281273,0.467752,0.769576,-0.842172,-0.016319,...,-0.395060,-0.099798,0.182876,1.345719,-0.003918,-0.298413,-0.437227,0.295602,0.653659,50
2,-0.252251,0.010676,0.076797,0.099302,-0.375454,-0.351671,0.686118,1.080516,-1.132174,-0.041633,...,-0.556308,-0.126104,0.228847,1.901903,-0.015765,-0.429257,-0.626690,0.397254,0.878924,153
3,-0.127534,-0.005729,0.055268,0.058820,-0.153877,-0.155063,0.276997,0.430063,-0.494674,0.002302,...,-0.253034,-0.068615,0.122414,0.793959,-0.009940,-0.175189,-0.271280,0.160464,0.378234,532
4,-0.185900,0.016835,0.078228,0.093165,-0.280630,-0.238812,0.495225,0.759293,-0.837643,-0.002351,...,-0.367155,-0.101637,0.170954,1.345756,-0.012862,-0.321034,-0.412941,0.271825,0.624616,679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.084134,0.009743,0.034356,0.026880,-0.079086,-0.087609,0.162613,0.252612,-0.292554,0.000389,...,-0.125829,-0.011673,0.083575,0.494027,-0.010611,-0.119213,-0.146132,0.086891,0.245972,19783
9996,-0.040230,-0.008888,0.025553,0.007989,-0.043686,-0.032440,0.068984,0.139844,-0.143229,-0.008378,...,-0.089659,-0.021681,0.025719,0.258108,-0.013732,-0.078491,-0.096725,0.057965,0.129235,17471
9997,-0.037533,-0.011593,-0.001954,0.024347,-0.057215,-0.057020,0.096433,0.132021,-0.165587,-0.001471,...,-0.092375,-0.026950,0.034231,0.267395,0.016119,-0.062557,-0.088470,0.069760,0.150359,3167
9998,-0.050234,-0.004495,0.025407,0.033631,-0.038754,-0.062975,0.089112,0.147673,-0.156655,-0.017144,...,-0.063009,-0.021668,0.015164,0.226488,-0.008969,-0.052724,-0.077009,0.069597,0.100324,12609


In [19]:
data1 = pd.DataFrame(data = degree_centrality.values(), index=degree_centrality.keys(), columns=["degree_centrality"]).reset_index().rename(columns={'index': 'node'})
data2 = pd.DataFrame(data = degree_centrality.values(), index=degree_centrality.keys(), columns=["Closeness_centrality"]).reset_index().rename(columns={'index': 'node'})

In [20]:
data = data0.merge(data1, on=['node']).merge(data2, on=['node'])

In [21]:
data.columns = data.columns.astype(str)

### train test split

In [22]:
train_df, test_df = train.merge(data, on=['node']), test.merge(data, on=['node'])
X_train, y_train, X_test, y_test = train_df.drop('target', axis=1), train_df['target'], test_df.drop('target', axis=1), test_df['target']

### ML on tabular data

In [23]:
def f_metrics(y_train, y_train_pred, y_test, y_test_pred) -> pd.DataFrame:
    MAE_train, MAE_test = median_absolute_error(y_train, y_train_pred), median_absolute_error(y_test, y_test_pred) # mae
    MSE_train, MSE_test = mean_squared_error(y_train, y_train_pred, squared=False), mean_squared_error(y_test, y_test_pred,  squared=True) # MSE
    MAPE_train, MAPE_test = mean_absolute_percentage_error(y_train, y_train_pred), mean_absolute_percentage_error(y_test, y_test_pred) # MAPE
    return(pd.DataFrame({'MAE':[MAE_train, MAE_test], 'MSE':[MSE_train,MSE_test], 'MAPE':[MAPE_train,MAPE_test]}, index=['Train','Test']))

In [24]:
regressor = DecisionTreeRegressor(min_samples_split=4, random_state=42)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(min_samples_split=4, random_state=42)

In [25]:
# extracted_MSEs = regressor.tree_.impurity # The Hidden magic is HERE

# for idx, MSE in enumerate(regressor.tree_.impurity):
#     print("Node {} has MSE {}".format(idx, MSE))

In [26]:
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

In [27]:
f_metrics(y_train, y_train_pred, y_test, y_test_pred)

,MAE,MSE,MAPE
Train,0.002403,0.018787,0.013026
Test,0.049057,0.026656,0.164524
